<a href="https://colab.research.google.com/github/shreyammb/INNOV8TIGERS/blob/main/ROADPROCESSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio scipy fiona

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geopandas as gpd
import fiona
import os
import rasterio
from rasterio import features
import numpy as np
from scipy.ndimage import distance_transform_edt
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio.io import MemoryFile

In [ ]:
in_gpkg = "/content/drive/MyDrive/bom/bom-roads/bom-unclassified.gpkg"
out_gpkg = "/content/drive/MyDrive/bom/bom-roads/bom-unclassified.gpkg"

layers = fiona.listlayers(in_gpkg)
print("Layers found:", layers)

for layer in layers:
    if "lines" in layer.lower():
        print(f"Keeping layer: {layer}")
        gdf = gpd.read_file(in_gpkg, layer=layer)
        gdf.to_file(out_gpkg, layer=layer, driver="GPKG")

print(f"Cleaned file written to: {out_gpkg}")

In [ ]:
sat_raster_path = "/content/drive/MyDrive/bom/bom_GLCM_B4.tif"
road_folder = "/content/drive/MyDrive/bom/bom-roads"
out_tif = "/content/drive/MyDrive/bom/road_distance_stack.tif"



road_files = {
    "primary": ("bom-primary.gpkg", "highway_primary"),
    "secondary": ("bom-secondary.gpkg", "highway_secondary"),
    "tertiary": ("bom-tertiary.gpkg", "highway_tertiary"),
    "unclassified": ("bom-unclassified.gpkg", "highway_unclassified"),
    "residential": ("bom-residential.gpkg", "highway_residential"),
}

with rasterio.open(sat_raster_path) as src:
    profile = src.profile
    transform = src.transform
    crs = src.crs
    height, width = src.height, src.width

profile.update(
    dtype="float32",
    count=len(road_files),
    compress="lzw"
)

bands = []


for i, (name, (filename, layer)) in enumerate(road_files.items(), start=1):
    road_path = os.path.join(road_folder, filename)
    print(f"[{i}/{len(road_files)}] Processing {name} roads from {layer}: {road_path}")

    roads = gpd.read_file(road_path, layer=layer)
    if roads.crs != crs:
        roads = roads.to_crs(crs)

    shapes = ((geom, 1) for geom in roads.geometry if geom is not None)
    road_raster = features.rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0,
        all_touched=True,
        dtype="uint8"
    )

    dist_px = distance_transform_edt(road_raster == 0)

    res_x, res_y = transform.a, -transform.e
    dist_m = dist_px * np.mean([abs(res_x), abs(res_y)])

    bands.append(dist_m.astype("float32"))

with rasterio.open(out_tif, "w", **profile) as dst:
    for i, dist_band in enumerate(bands, start=1):
        dst.write(dist_band, i)

print(f"Saved distance raster stack to: {out_tif}")


In [ ]:
!gdal_merge.py -v -o /content/drive/MyDrive/bom/merged.tif \
  -of GTiff \
  /content/drive/MyDrive/bom/A.tif \
  /content/drive/MyDrive/bom/B.tif